# 0- Imports

In [1]:
# Install dependencies
#!pip install -q amplpy

In [2]:
# Google Colab & Kaggle integration
from amplpy import AMPL, tools
ampl = tools.ampl_notebook(
    modules=["coin", "highs", "gokestrel", "gurobi"], # modules to install
    license_uuid="default", # license to use
    g=globals()) # instantiate AMPL object and register magics

AMPL Development Version 20240404 (MSVC 19.38.33135.0, 64-bit)
Demo license with maintenance expiring 20260131.
Using license file "c:\Users\canda\AppData\Local\Programs\Python\Python311\Lib\site-packages\ampl_module_base\bin\ampl.lic".



In [3]:
ampl.option["solver"] = "gurobi"

# 1- Modelagem

## Exemplo 1

In [4]:
%%ampl_eval
reset;

In [8]:
%%writefile exemplo_1.mod

# Parâmetros
param np; # numero de produtos
param nm; # numero de maquinas
set P = 1..np; # conjunto de produtos
set M = 1..nm; # conjunto de maquinas
param preco{P}; # preco de venda dos produtos
param custo{M}; # custo/hora das maquinas
param disp{M}; # disponibilidade das maquinas
param tempo{M,P}; # tempo gasto pelo produto j na maquina i

#Variáveis de decisão
var x{P} integer >= 0; # quantidade de produtos a produzir
var venda = sum{j in P} preco[j]*x[j];
var gasto = sum{i in M} custo[i]*(sum{j in P} tempo[i,j]*x[j])/60;

#Função-objetivo
maximize L: venda - gasto;
s.t. R1 {i in M}: sum{j in P} (tempo[i,j]*x[j])/60 <= disp[i];

Writing exemplo_1.mod


Valores dos parâmetros

In [6]:
%%writefile exemplo_1.dat
param np := 5;
param nm := 3;
param preco := 1 35 2 40 3 55 4 25 5 50;
param custo := 1 20 2 15 3 35;
param disp := 1 700 2 580 3 800;
param tempo:
1 2 3 4 5 :=
1 2 5 7 6 4
2 6 2 4 2 5
3 4 3 2 1 3;

Overwriting exemplo_1.dat


In [10]:
%%ampl_eval
reset;
model exemplo_1.mod;
data exemplo_1.dat;
solve;

# exibir valor das variáveis de decisão
display x;

Gurobi 11.0.1:Gurobi 11.0.1: optimal solution; objective 392024.75
2 simplex iterations
1 branching node
absmipgap=16.4265, relmipgap=4.19016e-05
x [*] :=
1     0
2  4163
3     1
4     0
5  5294
;



## Bin-packing

Modelagem matemática 

$$
\begin{aligned}
\text{minimize} \quad &\sum_{j = 1}^{m} y_{j} \\
\text{subject to} \quad &\sum_{i = 1}^{n} w_{i} x_{i j} \leq C y_{j} \quad \forall j \in \{1,...,m\} \\
&\sum_{j = 1}^{m} x_{i j} \geq 1 \quad \forall i \in \{1,...,n\} \\
&x_{i j} \in \{0,1\} \quad \forall i \in \{1,...,n\}, j \in \{1,...,m\} \\
&y_{j} \in \{0,1\} \quad \forall j \in \{1,...,m\}
\end{aligned}
$$

In [12]:
%%ampl_eval
reset;

In [13]:
%%ampl_eval
param n;
param C;

suffix master IN, binary;
suffix block IN, integer;

set I = 1..n ordered;
param w {I} > 0;
param maxVal := max {i in I} w[i];
param maxbins := ceil(n / floor(C / maxVal));

set J = 1..maxbins;

var x {I,J} binary;
var y {J} binary;

minimize Cost:  sum {j in J} y[j];

subject to b_Capacity {j in J}:
   sum {i in I} w[i] * x[i,j] <= C * y[j] suffix block j;

subject to m_Allocate {i in I}:
   sum {j in J} x[i,j] >= 1 suffix master 1;
